# Indice 3: Proporción de casos notificados con testeo oportuno

## Objetivo: evaluar la oportunidad de testeo en personas con indicación de RT-PCR

### Meta: >90%

**Periodicidad:** Semanal

**Escala territorial:** comunal, regional, nacional

**Fuente de información:** base EPIVIGILA actualizada con los reportes al día de la determinación del indicador

### Cálculo del indicador a nivel comunal

$\frac{No.\:notificados\:sospechosos\:c/test\:por\:comuna\: de\:residencia\:semana\:x}{No.\:total\:notificados\:sospechosos\:c/test\:por\:comuna\:de\: residencia\:semana\:x}$

### Cálculo del indicador a nivel regional

$\frac{No.\:notificados\:sospechosos\:c/test\:por\:regió\: de\:residencia\:semana\:x}{No.\:total\:notificados\:sospechosos\:c/test\:por\:región\:de\:residencia\:semana\:x}$

**Observación**

- Se excluyen los registros con etapa clínica probable 

## Importación de datos y librerias

In [1]:
# Importa librerias necesarias para el análisis

import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
# Importa base de datos generada mediante merging 

data = pd.read_csv("data/data_con_establecimiento_cp.csv", 
                   low_memory=False
                  )

In [3]:
# Filtra datos obtenidos solo para pacientes con residencia en la Región de Aysén

data_aysen = data[data['region_residencia'] == 'Región de Aysén del General Carlos Ibáñez del Campo']

data_aysen.head(1)

,Unnamed: 0,id_formulario_eno,numero_folio,id_enfermedad_eno,enfermedad_notificada,semana_epidemiologica,fecha_notificacion,etapa_clinica,establecimiento_salud,region,...,fecha_resultado_otro_cultivo_3,factor_riesgo_caso_confirmado,factor_riesgo_caso_sospechoso,factor_riesgo_viaje_internacional,factor_riesgo_viaje_nacional,ningun_factor_riesgo,test_antigeno_sars_positivo,tac_torax_covid,comuna_seremi_xi,provincia_seremi_xi
0,0,100166,50166,50,Coronavirus (COVID-19),11,2020-03-12,DESCARTADA,Hospital Regional (Coihaique),Región de Aysén del General Carlos Ibáñez del ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coyhaique,Coyhaique


## Variables de importancia


|Variable|Tipo|Descripción|Observación|
|---|---|---|---|
|`region_residencia`| Text | Residencia registrada | Variable utilizada pero no mostrada
|`numero_folio`|Numeric| id | el número de folio se mantendrá por trazabilidad
|`comuna_residencia`|Text| City | Comuna de residencia informada
|`tipo_identificacion`|Text| Tipo de documento de identidad | Carnet o Pasaporte
|`identificacion_paciente`|Numeric|Rol Unico Nominal o Numero de pasaporte| |
|`fecha_notificacion`|DateTime| Fecha de notificación| DateTime en formato `%Y-%m-%d` |
|`semana_epidemiologica`|Numeric|Semana epidemiologica|valores de 1 al 52|
|`etapa_clinica`|Text|`SOSPECHA` o `BUSQUEDA A`| Debe ser filtrada para evitar valores `DESCARTADA` |
|`resultado_pcr_1`|Text||
|`fecha_resultado_pcr_1`|DateTime||
|`fecha_toma_muestra_1`|DateTime||
|`resultado_pcr_2`|Text||
|`fecha_resultado_pcr_2`|DateTime||
|`fecha_toma_muestra_2`|DateTime||
|`resultado_pcr_3`|Text||
|`fecha_resultado_pcr_3`|DateTime||
|`fecha_toma_muestra_3`|DateTime||
|`establecimiento_salud`|Text|Establecimiento de salud que reporta caso||
|`comuna_seremi_xi`|Text|Comuna identificada a traves de establecimiento_salud||
|`provincia_seremi_xi`|Text|Comuna identificada a traves de establecimiento_salud||




In [6]:
# Selecciona variables de importancia


data_aysen_clean =  data_aysen[[
                                'numero_folio',
                                'comuna_residencia',
                                'fecha_notificacion',
                                'semana_epidemiologica',
                                'etapa_clinica',
                                'resultado_pcr_1',
                                'resultado_pcr_2',
                                'resultado_pcr_3',
                                'fecha_toma_muestra_1',
                                'fecha_toma_muestra_2',
                                'fecha_toma_muestra_3',
                                'fecha_resultado_pcr_1',
                                'fecha_resultado_pcr_2',
                                'fecha_resultado_pcr_3',
                                'establecimiento_salud',
                                'comuna_seremi_xi',
                                'provincia_seremi_xi'
                               ]]


In [10]:
# Filtra etapa_clinica SOSPECHA

data_aysen_clean = data_aysen_clean[data_aysen_clean['etapa_clinica']=='SOSPECHA']

In [11]:
# Transforma Fechas a formato DateTime 

data_aysen_clean['fecha_notificacion'] = pd.to_datetime(data_aysen_clean['fecha_notificacion']) 

data_aysen_clean['fecha_toma_muestra_1'] = pd.to_datetime(data_aysen_clean['fecha_toma_muestra_1'])

#data_aysen_clean['fecha_toma_muestra_2'] = pd.to_datetime(data_aysen_clean['fecha_toma_muestra_2'])

data_aysen_clean['fecha_toma_muestra_3'] = pd.to_datetime(data_aysen_clean['fecha_toma_muestra_3'])

data_aysen_clean['fecha_resultado_pcr_1'] = pd.to_datetime(data_aysen_clean['fecha_resultado_pcr_1'])

data_aysen_clean['fecha_resultado_pcr_2'] = pd.to_datetime(data_aysen_clean['fecha_resultado_pcr_2'])

data_aysen_clean['fecha_resultado_pcr_3'] = pd.to_datetime(data_aysen_clean['fecha_resultado_pcr_3'])


## Cálculo de indicadores

In [12]:
semana_x = 33

year = 2021

# Filtra datos para semana_x y año (year)

data_semx = data_aysen_clean[(data_aysen_clean['fecha_notificacion'].dt.year == year)&(data_aysen_clean['semana_epidemiologica']== semana_x)]

data_semx.sample(10)

,numero_folio,comuna_residencia,fecha_notificacion,semana_epidemiologica,etapa_clinica,resultado_pcr_1,resultado_pcr_2,resultado_pcr_3,fecha_toma_muestra_1,fecha_toma_muestra_2,fecha_toma_muestra_3,fecha_resultado_pcr_1,fecha_resultado_pcr_2,fecha_resultado_pcr_3,establecimiento_salud,comuna_seremi_xi,provincia_seremi_xi
72072,16062201,Chile Chico,2021-08-16,33,SOSPECHA,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaT,NaT,Hospital Dr. Leopoldo Ortega R. (Chile Chico),Chile Chico,General Carrera
68270,16246588,Aisén,2021-08-21,33,SOSPECHA,NEGATIVO,NaN,NEGATIVO,2021-04-20,NaN,2021-08-21,2021-04-21,NaT,2021-08-22,Hospital de Puerto Aysén,Aysen,Aysen
68111,16095328,Aisén,2021-08-16,33,SOSPECHA,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaT,NaT,Hospital de Puerto Aysén,Aysen,Aysen
22085,16242643,Coyhaique,2021-08-21,33,SOSPECHA,NaN,NEGATIVO,NEGATIVO,2021-08-21,2021-07-05,2021-08-21,NaT,2021-07-06,2021-08-22,Hospital Regional (Coihaique),Coyhaique,Coyhaique
72079,16108238,Chile Chico,2021-08-17,33,SOSPECHA,NEGATIVO,NaN,NaN,2021-08-11,NaN,NaT,2021-08-13,NaT,NaT,Hospital Dr. Leopoldo Ortega R. (Chile Chico),Chile Chico,General Carrera
22004,16224822,Coyhaique,2021-08-20,33,SOSPECHA,NaN,NEGATIVO,NaN,2021-08-20,2021-08-20,NaT,NaT,2021-08-21,NaT,Hospital Regional (Coihaique),Coyhaique,Coyhaique
72083,16121909,Chile Chico,2021-08-18,33,SOSPECHA,NEGATIVO,NaN,NaN,2021-05-31,NaN,NaT,2021-06-03,NaT,NaT,Hospital Dr. Leopoldo Ortega R. (Chile Chico),Chile Chico,General Carrera
22081,16241400,Coyhaique,2021-08-21,33,SOSPECHA,NaN,NEGATIVO,NaN,2021-08-21,2021-08-21,NaT,NaT,2021-08-22,NaT,Hospital Regional (Coihaique),Coyhaique,Coyhaique
72116,16247397,Chile Chico,2021-08-21,33,SOSPECHA,NEGATIVO,NaN,NaN,2021-08-02,NaN,NaT,2021-08-04,NaT,NaT,Hospital Dr. Leopoldo Ortega R. (Chile Chico),Chile Chico,General Carrera
68267,16244368,Aisén,2021-08-21,33,SOSPECHA,NaN,NEGATIVO,NaN,2021-08-21,2021-08-21,NaT,NaT,2021-08-22,NaT,Hospital de Puerto Aysén,Aysen,Aysen


## Indicador oportunidad de testeo

In [19]:
# Filtra casos con al menos un resultado PCR

data_semx_wpcr = data_semx[(data_semx['resultado_pcr_1'].notnull())|(data_semx['resultado_pcr_2'].notnull())|(data_semx['resultado_pcr_3'].notnull())]

data_semx_wpcr.sample(5)

,numero_folio,comuna_residencia,fecha_notificacion,semana_epidemiologica,etapa_clinica,resultado_pcr_1,resultado_pcr_2,resultado_pcr_3,fecha_toma_muestra_1,fecha_toma_muestra_2,fecha_toma_muestra_3,fecha_resultado_pcr_1,fecha_resultado_pcr_2,fecha_resultado_pcr_3,establecimiento_salud,comuna_seremi_xi,provincia_seremi_xi
21921,16188728,Coyhaique,2021-08-19,33,SOSPECHA,NaN,NEGATIVO,NaN,2021-08-19,2021-08-19,NaT,NaT,2021-08-20,NaT,Hospital Regional (Coihaique),Coyhaique,Coyhaique
21934,16192476,Coyhaique,2021-08-19,33,SOSPECHA,NaN,NEGATIVO,NaN,2021-08-19,2021-08-19,NaT,NaT,2021-08-20,NaT,Hospital Regional (Coihaique),Coyhaique,Coyhaique
82398,16227567,Coyhaique,2021-08-20,33,SOSPECHA,NEGATIVO,NaN,NaN,2021-07-18,NaN,NaT,2021-07-19,NaT,NaT,Consultorio Víctor Domingo Silva,Coyhaique,Coyhaique
22029,16229751,Coyhaique,2021-08-20,33,SOSPECHA,NEGATIVO,NaN,NEGATIVO,2021-08-02,2021-08-20,2021-08-20,2021-08-03,NaT,2021-08-21,Hospital Regional (Coihaique),Coyhaique,Coyhaique
72115,16244176,Chile Chico,2021-08-21,33,SOSPECHA,NEGATIVO,NaN,NaN,2021-06-30,NaN,NaT,2021-07-01,NaT,NaT,Hospital Dr. Leopoldo Ortega R. (Chile Chico),Chile Chico,General Carrera


In [18]:
data_semx_wpcr.shape[0]

98

In [15]:
data_semx.shape[0]

127

In [20]:
print('Indicador de oportunidad de testeo regional: ' + str(data_semx_wpcr.shape[0]/data_semx.shape[0]*100))

Indicador de oportunidad de testeo regional: 77.16535433070865


In [21]:
# Agrega 52 semanas a las semanas de 2021 y crea semana_epidemiologica_21

data_aysen_clean1 = data_aysen_clean[data_aysen_clean['fecha_notificacion']<=pd.to_datetime('2021-01-02')]
data_aysen_clean2 = data_aysen_clean[data_aysen_clean['fecha_notificacion']>=pd.to_datetime('2021-01-03')]


data_aysen_clean1['semana_epidemiologica_21'] = data_aysen_clean1['semana_epidemiologica']
data_aysen_clean2['semana_epidemiologica_21'] = data_aysen_clean2['semana_epidemiologica']+53

data_aysen_clean_21 = data_aysen_clean1.append(data_aysen_clean2, ignore_index=True)


<ipython-input-21-cd20b4640465>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean1['semana_epidemiologica_21'] = data_aysen_clean1['semana_epidemiologica']
<ipython-input-21-cd20b4640465>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean2['semana_epidemiologica_21'] = data_aysen_clean2['semana_epidemiologica']+53


In [24]:
data_aysen_clean_21.sample(40)

,numero_folio,comuna_residencia,fecha_notificacion,semana_epidemiologica,etapa_clinica,resultado_pcr_1,resultado_pcr_2,resultado_pcr_3,fecha_toma_muestra_1,fecha_toma_muestra_2,fecha_toma_muestra_3,fecha_resultado_pcr_1,fecha_resultado_pcr_2,fecha_resultado_pcr_3,establecimiento_salud,comuna_seremi_xi,provincia_seremi_xi,semana_epidemiologica_21
770,10540038,Coyhaique,2021-04-07,14,SOSPECHA,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaT,NaT,Ejercicio libre de la profesión XI región,Ejercicio Libre de la Profesión,Ejercicio Libre de la Profesión,67
733,15808193,Cisnes,2021-08-09,32,SOSPECHA,NEGATIVO,NaN,NaN,2021-05-27,NaN,NaT,2021-05-28,NaT,NaT,Hospital Dr. Jorge Ibar (Cisnes),Cisnes,Aysen,85
210,6797118,Coyhaique,2021-01-12,2,SOSPECHA,NaN,NaN,NaN,2021-01-12,NaN,NaT,NaT,NaT,NaT,Hospital Regional (Coihaique),Coyhaique,Coyhaique,55
352,13856610,Coyhaique,2021-06-16,24,SOSPECHA,NEGATIVO,NaN,NaN,2021-06-03,2021-06-16,NaT,2021-06-06,NaT,NaT,Hospital Regional (Coihaique),Coyhaique,Coyhaique,77
153,1774797,Coyhaique,2020-08-11,33,SOSPECHA,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaT,NaT,Centro Clínico Militar Coyhaique,Coyhaique,Coyhaique,33
501,10506678,Guaitecas,2021-04-07,14,SOSPECHA,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaT,NaT,Posta de Salud Rural Melinka,Guaitecas,Aysen,67
755,7499542,Coyhaique,2021-01-27,4,SOSPECHA,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaT,NaT,Ejercicio libre de la profesión XI región,Ejercicio Libre de la Profesión,Ejercicio Libre de la Profesión,57
729,15269983,Cisnes,2021-07-23,29,SOSPECHA,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaT,NaT,Hospital Dr. Jorge Ibar (Cisnes),Cisnes,Aysen,82
50,5338267,Coyhaique,2020-12-05,49,SOSPECHA,NaN,NaN,NaN,2020-12-05,NaN,NaT,NaT,NaT,NaT,Hospital Regional (Coihaique),Coyhaique,Coyhaique,49
764,8051766,Coyhaique,2021-02-09,6,SOSPECHA,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaT,NaT,Ejercicio libre de la profesión XI región,Ejercicio Libre de la Profesión,Ejercicio Libre de la Profesión,59


In [26]:
# Filtra casos SOSPECHOSOS con al menos un resultado PCR

data_aysen_clean_21_wpcr = data_aysen_clean_21[(data_aysen_clean_21['resultado_pcr_1'].notnull())|(data_aysen_clean_21['resultado_pcr_2'].notnull())|(data_aysen_clean_21['resultado_pcr_3'].notnull())]

data_aysen_clean_21_wpcr.sample(5)

,numero_folio,comuna_residencia,fecha_notificacion,semana_epidemiologica,etapa_clinica,resultado_pcr_1,resultado_pcr_2,resultado_pcr_3,fecha_toma_muestra_1,fecha_toma_muestra_2,fecha_toma_muestra_3,fecha_resultado_pcr_1,fecha_resultado_pcr_2,fecha_resultado_pcr_3,establecimiento_salud,comuna_seremi_xi,provincia_seremi_xi,semana_epidemiologica_21
648,10296555,Chile Chico,2021-04-01,13,SOSPECHA,NaN,NEGATIVO,NaN,NaT,2021-03-22,NaT,NaT,2021-03-24,NaT,Hospital Dr. Leopoldo Ortega R. (Chile Chico),Chile Chico,General Carrera,66
300,12053576,Coyhaique,2021-05-10,19,SOSPECHA,NEGATIVO,NaN,NaN,2021-04-16,2021-05-10,NaT,2021-04-16,NaT,NaT,Hospital Regional (Coihaique),Coyhaique,Coyhaique,72
295,11856520,Coyhaique,2021-05-05,18,SOSPECHA,NEGATIVO,NaN,NaN,2021-03-15,2021-05-05,NaT,2021-03-15,NaT,NaT,Hospital Regional (Coihaique),Coyhaique,Coyhaique,71
386,16198274,Coyhaique,2021-08-20,33,SOSPECHA,NaN,NEGATIVO,NaN,2021-08-20,2021-08-20,NaT,NaT,2021-08-20,NaT,Hospital Regional (Coihaique),Coyhaique,Coyhaique,86
698,11710105,Coyhaique,2021-04-30,17,SOSPECHA,NaN,NEGATIVO,NaN,NaT,2021-04-30,NaT,NaT,2021-05-02,NaT,Consultorio Víctor Domingo Silva,Coyhaique,Coyhaique,70


In [28]:
# Filtra casos SOSPECHOSOS sin resultado PCR

data_aysen_clean_21_wo_pcr = data_aysen_clean_21[data_aysen_clean_21['resultado_pcr_1'].isnull()]

data_aysen_clean_21_wo_pcr.sample(5)


,numero_folio,comuna_residencia,fecha_notificacion,semana_epidemiologica,etapa_clinica,resultado_pcr_1,resultado_pcr_2,resultado_pcr_3,fecha_toma_muestra_1,fecha_toma_muestra_2,fecha_toma_muestra_3,fecha_resultado_pcr_1,fecha_resultado_pcr_2,fecha_resultado_pcr_3,establecimiento_salud,comuna_seremi_xi,provincia_seremi_xi,semana_epidemiologica_21
254,10194667,Coyhaique,2021-03-31,13,SOSPECHA,NaN,NO CONCLUYENTE,NaN,2021-03-31,2021-03-31,NaT,NaT,2021-04-01,NaT,Hospital Regional (Coihaique),Coyhaique,Coyhaique,66
70,4422082,Coyhaique,2020-11-09,46,SOSPECHA,NaN,NaN,NaN,2020-11-09,NaN,NaT,NaT,NaT,NaT,SAPU Dr. Alejandro Gutiérrez,Coyhaique,Coyhaique,46
135,123774,Río Ibáñez,2020-04-14,16,SOSPECHA,NaN,NaN,NaN,NaT,NaN,NaT,NaT,NaT,NaT,Centro Clínico Militar Coyhaique,Coyhaique,Coyhaique,16
246,9555448,Coyhaique,2021-03-18,11,SOSPECHA,NaN,NO CONCLUYENTE,NaN,2021-03-18,2021-03-18,NaT,NaT,2021-03-19,NaT,Hospital Regional (Coihaique),Coyhaique,Coyhaique,64
222,7347779,Coyhaique,2021-01-24,4,SOSPECHA,NaN,NaN,NaN,2021-01-24,NaN,NaT,NaT,NaT,NaT,Hospital Regional (Coihaique),Coyhaique,Coyhaique,57


In [72]:
data_aysen_clean_21_wo_pcr['oportunidad'] = False

data_aysen_clean_21_wpcr['oportunidad'] = True

<ipython-input-72-7ded97baaea8>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-72-7ded97baaea8>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [78]:
data_aysen_clean_21_wpcr.sample(5)

,numero_folio,comuna_residencia,fecha_notificacion,semana_epidemiologica,etapa_clinica,resultado_pcr_1,resultado_pcr_2,resultado_pcr_3,fecha_toma_muestra_1,fecha_toma_muestra_2,fecha_toma_muestra_3,fecha_resultado_pcr_1,fecha_resultado_pcr_2,fecha_resultado_pcr_3,establecimiento_salud,comuna_seremi_xi,provincia_seremi_xi,semana_epidemiologica_21,oportunidad,not_con_pcr
522,16121052,Cisnes,2021-08-18,33,SOSPECHA,NEGATIVO,NaN,NEGATIVO,2021-06-09,NaN,2021-08-18,2021-06-10,NaT,2021-08-19,Posta de Salud Rural Puyuhuapi,Cisnes,Aysen,86,True,True
475,16259537,Coyhaique,2021-08-22,34,SOSPECHA,NEGATIVO,NaN,NaN,2021-07-31,2021-08-22,NaT,2021-08-01,NaT,NaT,SAPU Dr. Alejandro Gutiérrez,Coyhaique,Coyhaique,87,True,True
460,16236948,Coyhaique,2021-08-21,33,SOSPECHA,NaN,NEGATIVO,NaN,2021-08-21,2021-08-21,NaT,NaT,2021-08-22,NaT,SAPU Dr. Alejandro Gutiérrez,Coyhaique,Coyhaique,86,True,True
423,16234962,Coyhaique,2021-08-21,33,SOSPECHA,NaN,NEGATIVO,NaN,2021-08-21,2021-08-21,NaT,NaT,2021-08-22,NaT,Hospital Regional (Coihaique),Coyhaique,Coyhaique,86,True,True
305,12209609,Coyhaique,2021-05-13,19,SOSPECHA,NEGATIVO,NEGATIVO,NaN,2021-04-10,2021-04-19,2021-05-13,2021-04-11,2021-04-20,2021-05-13,Hospital Regional (Coihaique),Coyhaique,Coyhaique,72,True,True


In [74]:
# Agrega ambos datasets mediante append

data_oportunidad = data_aysen_clean_21_wo_pcr.append(data_aysen_clean_21_wpcr)

data_oportunidad.sample(5)

,numero_folio,comuna_residencia,fecha_notificacion,semana_epidemiologica,etapa_clinica,resultado_pcr_1,resultado_pcr_2,resultado_pcr_3,fecha_toma_muestra_1,fecha_toma_muestra_2,...,fecha_resultado_pcr_1,fecha_resultado_pcr_2,fecha_resultado_pcr_3,establecimiento_salud,comuna_seremi_xi,provincia_seremi_xi,semana_epidemiologica_21,oportunidad,not_sin_pcr,not_con_pcr
168,3309851,Coyhaique,2020-10-05,41,SOSPECHA,NaN,NaN,NaN,NaT,NaN,...,NaT,NaT,NaT,Ejercicio libre de la profesión XI región,Ejercicio Libre de la Profesión,Ejercicio Libre de la Profesión,41,False,False,NaN
651,10971411,Chile Chico,2021-04-15,15,SOSPECHA,NaN,NaN,NaN,NaT,NaN,...,NaT,NaT,NaT,Hospital Dr. Leopoldo Ortega R. (Chile Chico),Chile Chico,General Carrera,68,False,False,NaN
655,12489122,Chile Chico,2021-05-20,20,SOSPECHA,NaN,NaN,NaN,NaT,NaN,...,NaT,NaT,NaT,Hospital Dr. Leopoldo Ortega R. (Chile Chico),Chile Chico,General Carrera,73,False,False,NaN
383,16196510,Coyhaique,2021-08-20,33,SOSPECHA,NaN,NEGATIVO,NaN,2021-08-20,2021-08-20,...,NaT,2021-08-20,NaT,Hospital Regional (Coihaique),Coyhaique,Coyhaique,86,True,NaN,True
195,3082066,Coyhaique,2020-09-28,40,SOSPECHA,NaN,NaN,NaN,2020-09-28,NaN,...,NaT,NaT,NaT,Actividades gestionadas por la Dirección del S...,Coyhaique,Coyhaique,40,False,False,NaN


In [75]:
oportunidad = data_oportunidad.set_index('semana_epidemiologica_21').groupby(['semana_epidemiologica_21','oportunidad']).count()[['numero_folio']].rename(columns={'numero_folio':'casos'})

oportunidad

casos
semana_epidemiologica_21 oportunidad       
12                       False            1
13                       False            1
14                       False            1
15                       False            1
16                       False           27
...                                     ...
85                       True             7
86                       False           89
                         True            98
87                       False           34
                         True            32

[96 rows x 1 columns]

In [76]:
# Crea gráfico de Resultados PCR regional por semana epidemiologica

px.bar(oportunidad.reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'casos',
        color= 'oportunidad'
        
       )

In [77]:
# Crea gráfico de Resultados PCR regional por semana epidemiologica

px.line(oportunidad.reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'casos',
        color= 'oportunidad'
       )